In [2]:
print("X_train.shape:", X_train.shape)
print("X_test.shape: ", X_test.shape)


X_train.shape: (3689, 5)
X_test.shape:  (0, 5)


In [3]:
print("Mínima fecha:", df_model['fecha'].min())
print("Máxima fecha:", df_model['fecha'].max())

Mínima fecha: 2021-09-13 10:41:41
Máxima fecha: 2024-02-02 17:37:32


In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

# 0) Carga de datos
df = pd.read_csv("dataset_combinado.csv", parse_dates=['fecha'])

# 1) Filtrar sólo datos reales
df_real = df[df['usuario'].isin(['usuario_strong', 'usuario_weightlifting'])].copy()

# 2) Ordenar y crear la variable objetivo: próxima carga
df_real = df_real.sort_values(['usuario', 'ejercicio', 'fecha'])
df_real['peso_next'] = df_real.groupby(['usuario','ejercicio'])['peso (kg)'].shift(-1)

# 3) Crear features de lag y time-deltas
df_real['peso_prev'] = df_real.groupby(['usuario','ejercicio'])['peso (kg)'].shift(1)
df_real['reps_prev'] = df_real.groupby(['usuario','ejercicio'])['reps'].shift(1)
df_real['rir_prev']  = df_real.groupby(['usuario','ejercicio'])['rir'].shift(1)
df_real['delta_peso'] = df_real['peso (kg)'] - df_real['peso_prev']
df_real['dias_entre_sesiones'] = df_real.groupby(['usuario','ejercicio'])['fecha'].diff().dt.days

# 4) Eliminar filas con NaN en features u objetivo
features = ['peso_prev','reps_prev','rir_prev','delta_peso','dias_entre_sesiones']
df_model = df_real.dropna(subset=features + ['peso_next']).copy()

# 5) Split temporal: antes de 2024-01-01 → train; desde 2024-01-01 → test
split_date = '2024-01-03'
train = df_model[df_model['fecha'] < split_date]
test  = df_model[df_model['fecha'] >= split_date]

X_train, y_train = train[features], train['peso_next']
X_test,  y_test  = test[features],  test['peso_next']

print(f"Train shape: {X_train.shape}  Test shape: {X_test.shape}")

# 6) Entrenar modelo baseline
model = DecisionTreeRegressor(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 7) Predecir y evaluar
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f"MAE (solo datos reales): {mae:.2f} kg")

# 8) Ver primeras predicciones vs realidad
comparison = test[['usuario','ejercicio','fecha','peso (kg)','peso_next']].copy()
comparison['predicción'] = preds
print(comparison.head(5))


Train shape: (3503, 5)  Test shape: (186, 5)
MAE (solo datos reales): 1.45 kg
             usuario              ejercicio               fecha  peso (kg)  \
3821  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   
3822  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   
3823  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   
3824  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       25.0   
3837  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   

      peso_next  predicción  
3821       30.0   29.907807  
3822       30.0   29.907807  
3823       25.0   29.907807  
3824       30.0   29.907807  
3837       30.0   29.330357  


In [10]:
# Regla +2 % comparación
preds_rule = test['peso (kg)'] * 1.02
mae_rule = mean_absolute_error(y_test, preds_rule)
print(f"MAE regla +2 %: {mae_rule:.2f} kg")


MAE regla +2 %: 1.44 kg
